<a href="https://colab.research.google.com/github/nishkalavallabhi/practicalnlp/blob/V_2_0/Ch4/04%20Logistic%20Regression%20with%20doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NACHISS - DATA NEEDED

In [0]:
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
 
import sys
import numpy
import random
 
 
class TaggedLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
 
        flipped = {}
 
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
 
    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
 
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(TaggedDocument(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return(self.sentences)
 
    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return(shuffled)

In [0]:
print 'loading corpus'

sources = {'./../../../dataset/test-neg.txt': 'TEST_NEG',
                  './../../../dataset/test-pos.txt': 'TEST_POS',
                  './../../../dataset/train-neg.txt': 'TRAIN_NEG',
                  './../../../dataset/train-pos.txt': 'TRAIN_POS'}

loading corpus


In [0]:
sentences = TaggedLineSentence(sources)
model = Doc2Vec(min_count=1, window=10, vector_size=100, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())
for epoch in range(10):
    print 'epoch: ' + str(epoch)
    #model.train(sentences.sentences_perm())
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
 
model.save('./imdb.d2v')
model = Doc2Vec.load('./imdb.d2v')
 
print 'starting to train'
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)
for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[12500 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0
 
print 'starting to test'
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)
for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[12500 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

epoch: 0


/home/root1/.virtualenv/demos/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  import sys


epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
starting to train
starting to test


In [0]:
print 'fitting model'
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)
LogisticRegression(
    C=1.0, class_weight=None, dual=False, fit_intercept=True,
    intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)
 
print str(classifier.score(test_arrays, test_labels))

fitting model
0.62804
